<a href="https://colab.research.google.com/github/burntato/Twitter-emotion-detection-ml/blob/master/js08_uts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Bagaseto Yudistira Fiandra Putra<br>
NIM : 2041720244<br>
Kelas : TI-3D<br>

In [ ]:
# install packages ( google colab )
# pip install pandas
# pip install numpy
# pip install matplotlib
# pip install seaborn
# pip install sklearn
# pip install scipy
# pip install statsmodels

## Deteksi Emosi Pengguna Twitter

Deteksi emosi merupakan salah satu permasalahan yang dihadapi pada ***Natural Language Processing*** (NLP). Alasanya diantaranya adalah kurangnya dataset berlabel untuk mengklasifikasikan emosi berdasarkan data twitter. Selain itu, sifat dari data twitter yang dapat memiliki banyak label emosi (***multi-class***). Manusia memiliki berbagai emosi dan sulit untuk mengumpulkan data yang cukup untuk setiap emosi. Oleh karena itu, masalah ketidakseimbangan kelas akan muncul (***class imbalance***). Pada Ujian Tengah Semester (UTS) kali ini, Anda telah disediakan dataset teks twitter yang sudah memiliki label untuk beberapa kelas emosi. Tugas utama Anda adalah membuat model yang mumpuni untuk kebutuhan klasifikasi emosi berdasarkan teks.

### Informasi Data

Dataset yang akan digunakan adalah ****tweet_emotion.csv***. Berikut merupakan informasi tentang dataset yang dapat membantu Anda.

- Total data: 40000 data
- Label emosi: anger, boredom, empty, enthusiasm, fun, happiness, hate, love, neutral, relief, sadness, surprise, worry
- Jumlah data untuk setiap label tidak sama (***class imbalance***)
- Terdapat 3 kolom = 'tweet_id', 'sentiment', 'content'

### Penilaian UTS

UTS akan dinilai berdasaarkan 4 proses yang akan Anda lakukan, yaitu pra pengolahan data, ektraksi fitur, pembuatan model machine learning, dan evaluasi.

#### Pra Pengolahan Data

> **Perhatian**
> 
> Sebelum Anda melakukan sesuatu terhadap data Anda, pastikan data yang Anda miliki sudah "baik", bebas dari data yang hilang, menggunakan tipe data yang sesuai, dan sebagainya.
>

Data tweeter yang ada dapatkan merupakan sebuah data mentah, maka beberapa hal dapat Anda lakukan (namun tidak terbatas pada) yaitu,

1. Case Folding
2. Tokenizing
3. Filtering
4. Stemming

*CATATAN: PADA DATA TWITTER TERDAPAT *MENTION* (@something) YANG ANDA HARUS TANGANI SEBELUM MASUK KE TAHAP EKSTRAKSI FITUR*

#### Ekstrasi Fitur

Anda dapat menggunakan beberapa metode, diantaranya

1. Bag of Words (Count / TF-IDF)
2. N-gram
3. dan sebagainya

#### Pembuatan Model

Anda dibebaskan dalam memilih algoritma klasifikasi. Anda dapat menggunakan algoritma yang telah diajarkan didalam kelas atau yang lain, namun dengan catatan. Berdasarkan asas akuntabilitas pada pengembangan model machine learning, Anda harus dapat menjelaskan bagaimana model Anda dapat menghasilkan nilai tertentu.

#### Evaluasi

Pada proses evaluasi, minimal Anda harus menggunakan metric akurasi. Akan tetapi Anda juga dapat menambahkan metric lain seperti Recall, Precision, F1-Score, detail Confussion Metric, ataupun Area Under Curve (AUC).

### Lembar Pengerjaan
Lembar pengerjaan dimulai dari cell dibawah ini

In [ ]:
# default usage
import numpy as np
import pandas as pd

# data preprocessing
import re 

# feature extraction ( TF-IDF )
from sklearn.feature_extraction.text import TfidfVectorizer

# model selection
from sklearn.model_selection import train_test_split

# model training
from sklearn.naive_bayes import MultinomialNB

# model evaluation
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
# upload tweet_emotions.csv file ke google colab
# from google.colab import files

# uploaded = files.upload()

In [ ]:
df = pd.read_csv('data/tweet_emotions.csv')

df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


Pra pengolahan data / data preprocessing

Disini saya menggunakan metode case folding dan filtering untuk menghilangkan data yang tidak digunakan, seperti sentiment 'empty' dan tweet_id yang tidak terpakai.

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # ini untuk menghilangkan @
    text = re.sub(r'#', '', text) # ini untuk menghilangkan #
    text = re.sub(r'RT[\s]+', '', text) # ini untuk menghilangkan RT
    text = re.sub(r'https?:\/\/\S+', '', text) # ini untuk menghilangkan hyperlink
    
    return text

df['content'] = df['content'].apply(clean_text) # implementasi fungsi clean_text

df.head()

,tweet_id,sentiment,content
0,1956967341,empty,i know i was listenin to bad habit earlier a...
1,1956967666,sadness,layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends soon!
4,1956968416,neutral,we want to trade with someone who has houston...


In [ ]:
df = df[df['sentiment'] != 'empty']

df.head()

,tweet_id,sentiment,content
1,1956967666,sadness,layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends soon!
4,1956968416,neutral,we want to trade with someone who has houston...
5,1956968477,worry,re-pinging : why didn't you go to prom? bc my ...


In [ ]:
df = df.drop('tweet_id', axis=1)

df.head()

,sentiment,content
1,sadness,layin n bed with a headache ughhhh...waitin o...
2,sadness,funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends soon!
4,neutral,we want to trade with someone who has houston...
5,worry,re-pinging : why didn't you go to prom? bc my ...


In [ ]:
df['sentiment'].unique()

array(['sadness', 'enthusiasm', 'neutral', 'worry', 'surprise', 'love',
       'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger'],
      dtype=object)

Ekstraksi Fitur / Feature Extraction

In [ ]:
vectorizer = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.7, stop_words='english')

X = vectorizer.fit_transform(df['content']).toarray()

X.shape

(39173, 1000)

In [ ]:
y = df['sentiment']

y.shape

(39173,)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30)

In [ ]:
X_train.shape

(29379, 1000)

In [ ]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
classifier = MultinomialNB()

classifier.fit(X_train, y_train)

MultinomialNB()

Evaluasi / Evaluation

In [ ]:
y_pred = classifier.predict(X_test)

y_pred

array(['worry', 'neutral', 'worry', ..., 'neutral', 'love', 'neutral'],
      dtype='<U10')

In [ ]:
cm = confusion_matrix(y_test, y_pred)

print(cm)

[[   0    0    0    0    1    0    0   12    0    0    0   14]
 [   0    0    0    0    1    0    0   14    0    4    0   19]
 [   0    0    0    0   24    0    7   91    0    4    1   64]
 [   0    0    0    1   85    0   23  203    0    8    0  129]
 [   0    0    0    1  374    0  115  541    1   14    0  265]
 [   0    0    0    0    5    9    4  133    0   38    0  157]
 [   0    0    0    0  174    0  281  303    2   23    1  175]
 [   0    0    0    0  144    3   64 1332    0   43    3  570]
 [   0    0    0    0   39    0   17  174    4    3    0  105]
 [   0    0    0    0   39    3   21  404    0  201    1  609]
 [   0    0    0    0   59    0   29  231    1   20    4  200]
 [   0    0    0    0   89    2   50  772    2  146    1 1088]]


In [ ]:
print(classification_report(y_test, y_pred, zero_division=1))

              precision    recall  f1-score   support

       anger       1.00      0.00      0.00        27
     boredom       1.00      0.00      0.00        38
  enthusiasm       1.00      0.00      0.00       191
         fun       0.50      0.00      0.00       449
   happiness       0.36      0.29      0.32      1311
        hate       0.53      0.03      0.05       346
        love       0.46      0.29      0.36       959
     neutral       0.32      0.62      0.42      2159
      relief       0.40      0.01      0.02       342
     sadness       0.40      0.16      0.23      1278
    surprise       0.36      0.01      0.01       544
       worry       0.32      0.51      0.39      2150

    accuracy                           0.34      9794
   macro avg       0.55      0.16      0.15      9794
weighted avg       0.39      0.34      0.29      9794



In [ ]:
print("Akurasi : ", round(accuracy_score(y_test, y_pred) * 100, 2), "%")

Akurasi :  33.63 %


### Dibawah adalah metode lain yang saya coba untuk menambah akurasi model

In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='linear', random_state=0)

svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)

cm = confusion_matrix(y_test, y_pred)

print(cm)

print(classification_report(y_test, y_pred, zero_division=1))

print("Akurasi : ", round(accuracy_score(y_test, y_pred) * 100, 2), "%")

[[   0    0    0    0    4    2    1    8    0    1    0   11]
 [   0    0    0    0    2    2    1   14    0    8    0   11]
 [   0    0    0    0   40    1    6   83    0    6    2   53]
 [   0    0    0    6  134    3   21  160    0   16    1  108]
 [   0    0    0    5  519    2  129  420    9   21    7  199]
 [   0    0    0    0   12   59    4  119    2   33    1  116]
 [   0    0    0    2  227    1  364  224    5   26    4  106]
 [   0    0    0    2  215   10   76 1307    9   59    5  476]
 [   0    0    0    0   68    1   24  141   15    9    0   84]
 [   0    0    0    3   75   19   28  383    2  270    4  494]
 [   0    0    0    0   94    1   23  203    3   32   10  178]
 [   0    0    0    4  176   22   59  698    7  223   11  950]]
              precision    recall  f1-score   support

       anger       1.00      0.00      0.00        27
     boredom       1.00      0.00      0.00        38
  enthusiasm       1.00      0.00      0.00       191
         fun       0.27   

In [ ]:
# increasing SVC accuracy by using GridSearchCV

from sklearn.model_selection import GridSearchCV

parameters = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']},

                {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.5, 0.1, 0.01, 0.001, 0.0001]}]

grid_search = GridSearchCV(estimator = svm, param_grid = parameters, scoring = 'accuracy', cv = 10, n_jobs = 10)

grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_

best_parameters = grid_search.best_params_

print("Best Accuracy: {:.2f} %".format(best_accuracy*100))

print("Best Parameters:", best_parameters)

y_pred = grid_search.predict(X_test)

cm = confusion_matrix(y_test, y_pred)

print(cm)

print(classification_report(y_test, y_pred, zero_division=1))

print("Akurasi : ", round(accuracy_score(y_test, y_pred) * 100, 2), "%")

In [ ]:
# predicting new tweet

tweet = "Very cool tweet"

tweet = clean_text(tweet)

tweet = vectorizer.transform([tweet]).toarray()

classifier.predict(tweet)

array(['neutral'], dtype='<U10')

In [ ]:
# predicting an array of tweets

tweets = ["Very cool tweet", "I am so happy", "I am so sad", "I am so angry"]

tweets = [clean_text(tweet) for tweet in tweets]

tweets = vectorizer.transform(tweets).toarray()

classifier.predict(tweets)

array(['neutral', 'love', 'sadness', 'neutral'], dtype='<U10')